## 1 - create new db projects (sqlite,mysql)
## 1.1 - using python + peewee : 
##   2 - create Cateogry table (name)
##   3 - create Client table (first_name,last_name,phone,address,email)
##    4 - create Project table (name,duration(days),client(relation),category(relation))
##    4.1.2.3 - insert new (client , category , project )
##    5.1.2.3 - create dummy data using website (20 projects(file) , 5 category(file) , 20 client(file)) as json or csv 
##    6.1.2.3- insert data from file into db (category ,client , project)
## - answer this questions :
##    7 - how many project for each client
##    8 - how many project in each category
##    9 - how many project we have with duaration > 30 day
##   10- what is the first project to finish
##   11- what is the last project to finish

has context menu

## 1 - create new db projects (sqlite,mysql)
## 1.1 - using python + peewee : 

In [1]:
from peewee import *
db =MySQLDatabase(
    "projects",
    user = "root",
    password = "1988",
    host = "localhost"
)
db.connect()
print("connect")

connect


## 2 - create Cateogry table (name)

In [5]:
class Category(Model):
    id = AutoField()
    name = CharField()
    class Meta:
        database = db # create table on db 
db.create_tables([Category])

##   3 - create Client table (first_name,last_name,phone,address,email)

In [6]:
class Client(Model):
    id = AutoField()
    first_name = CharField()
    last_name = CharField()
    phone = TextField()
    address = TextField()
    email = TextField()
    class Meta:
        database = db # create table on db 
db.create_tables([Client])

## 4 - create Project table (name,duration(days),client(relation),category(relation))

In [7]:
class Project(Model):
    id = AutoField()
    name = CharField()
    duration_days = IntegerField()
    client = ForeignKeyField( Client , backref='Client')# client(relation)
    category = ForeignKeyField( Category , backref='Category') # category(relation)
    class Meta:
        database = db # create table on db 
db.create_tables([Project])

## 4 (4.1) - insert new (client)

In [8]:
client = [
    ['Ali' ,'aLi', '0176559632','Dresden','ali.ali@gmail.com'] , 
    ['Ali1' ,'aLi1', '0176559623','Dresden1','ali1.ali@gmail.com'] 
 ]
for x in client:
    Client.create(first_name=x[0],last_name=x[1],phone=x[2],address=x[3],email=x[4] )

## 4 (4.2) - insert new ( category )

In [9]:
category = [
    ['aLi_cateogry1'] , 
    ['aLi1_cateogry2'] 
 ]
for x in category:
    Category.create(name = x[0])

## 4 (4.3) - insert new ( project )

In [10]:
project = [
    ['Ali_ali1' , 1 , 1 , 1 ] , 
    ['Ali1_ali2' , 2 , 2 , 2] 
 ]
for x in project:
    f = Client.get(Client.id == x[2])
    c = Category.get(Category.id == x[3])
    Project.create(name=x[0],duration_days=x[1], client = f , category = c)

## 5.1.2.3 - create dummy data using website (20 projects(file) , 5 category(file) , 20 client(file)) as json or csv

## 5.2 -insert dummy file category(5-file)
## -6.2- insert data from file into db (category )

In [12]:
import json
f = open('category_DATA.json')
data = json.load(f)
 # insert 10 product using build insert 
for p in data:
      Category.create(**p)

## 5.3-insert dummy file client(20-file))
## - 6.3- insert data from file into db (client )

In [13]:
import json
f = open('client_DATA.json')
data = json.load(f)
for p in data:
      Client.create(**p)

## 5.1- insert dummy file projects(20-file)
## - 6.1- insert data from file into db (project)

In [16]:
import json
import random
f = open('project_DATA.json')
data = json.load(f)
for p in data:
    Project.create(
          name = p['name'],
          duration_days = p['duration_days'],
          # client = Client.get(p['client_id']),
          client = Client.get(p['client_id']) if p['client_id'] <= 22 else Client.get(random.randint(1,22)),
          category = Category.get(p['category_id']) if p['category_id'] <= 7 else Category.get(random.randint(1,7))

          # category = Category.get(p['category_id'])
          )

## 7 - how many project for each client

In [50]:
# data = db.execute_sql('''select name , count(id) as count 
# from project
# GROUP BY client_id ''')
# print(list(data))

data = db.execute_sql('''SELECT c.first_name AS client_name, COUNT(p.id) AS project_count
FROM Client c
LEFT JOIN Project p ON c.id = p.client_id
GROUP BY c.first_name; ''' )
print(list(data))

[('Ali', 3), ('Ali1', 3), ('Belva', 2), ('Belia', 2), ('Brian', 2), ('Gweneth', 2), ('Lenci', 2), ('Cacilia', 1), ('Bonnibelle', 1), ('Karolina', 1), ('Archibaldo', 1), ('Andrei', 1), ('Darcey', 1), ('Gonzales', 1), ('Myrah', 1), ('Cchaddie', 1), ('Benedicta', 1), ('Rocky', 1), ('Elisha', 1), ('Randi', 1), ('Jarvis', 0), ('Charisse', 0)]


## 8 - how many project in each category

In [49]:
# data = db.execute_sql('''select name , count(*) 
# from project
# GROUP BY category_id ''')
# print(list(data))

data = db.execute_sql('''SELECT c.name AS category_name, COUNT(p.id) AS project_count
FROM Category c
LEFT JOIN Project p ON c.id = p.category_id
GROUP BY c.name; ''' )
print(list(data))


[('aLi_cateogry1', 5), ('aLi1_cateogry2', 5), ('Electrical', 5), ('Drywall & Acoustical (MOB)', 2), ('Drywall & Acoustical (FED)', 5), ('Retaining Wall and Brick Pavers', 4), ('Prefabricated Aluminum Metal Canopies', 3)]


##  9 - how many project we have with duaration > 30 day

In [60]:
# data = db.execute_sql('''select * from project
# where duration_days > 10 ''')
# print(list(data))
query = Project.select().where(Project.duration_days > 10)
query.execute()
print(list(query))

[<Project: 20>, <Project: 21>, <Project: 22>, <Project: 23>, <Project: 24>, <Project: 25>, <Project: 26>, <Project: 27>, <Project: 28>, <Project: 29>]


## 10 - what is the first project to finish ( with peewee )

In [66]:
# ORDER BY category DESC-desc
# ORDER BY category ASC-asc
# query = Project.select().ORDER BY project DESC # what is the first project to finish
# query.execute()
# print(list(query))
data = Project.select().order_by(Project.name.asc())
for x in data:
    print(x.name)


Ali_ali1
Ali1_ali2
Aurillac Airport
Birmingham International Airport
Birmingham International Airport
Cambridge Bay Airport
Cambridge Bay Airport
Earl Henry Airport
Edmonton City Centre (Blatchford Field) Airport
Edmonton City Centre (Blatchford Field) Airport
Edson Airport
Edson Airport
Fair Isle Airport
Francisco BeltrÃ£o Airport
Hsinchu Air Base
Jask Airport
Kornasoren Airfield
Kratie Airport
Mondell Field
Mondell Field
New Castle Airport
Newport News Williamsburg International Airport
Norman Wells Airport
Okondja Airport
Ormoc Airport
Ormoc Airport
Sheremetyevo International Airport
Sheremetyevo International Airport
Wales Airport


## 11 - what is the last project to finish ( with peewee )

In [65]:
data = Project.select().order_by(Project.name.desc())
for x in data:
    print(x.name)

Wales Airport
Sheremetyevo International Airport
Sheremetyevo International Airport
Ormoc Airport
Ormoc Airport
Okondja Airport
Norman Wells Airport
Newport News Williamsburg International Airport
New Castle Airport
Mondell Field
Mondell Field
Kratie Airport
Kornasoren Airfield
Jask Airport
Hsinchu Air Base
Francisco BeltrÃ£o Airport
Fair Isle Airport
Edson Airport
Edson Airport
Edmonton City Centre (Blatchford Field) Airport
Edmonton City Centre (Blatchford Field) Airport
Earl Henry Airport
Cambridge Bay Airport
Cambridge Bay Airport
Birmingham International Airport
Birmingham International Airport
Aurillac Airport
Ali1_ali2
Ali_ali1
